# Nearest Neighbors Tutorial

The basic concept for **Nearest Neighbors Algorithms** (NNAs) is to classify a datum by finding one or more points that have similar features. The most similar points are called the **nearest neighbors**. Once found, the input datum can be put in the same class as its neighbors. We can also use this to predict the value of missing features for the input datum.

## K-Nearest Neighbors

The most commonly used NNA is **k-Nearest Neighbors,** in which the top $k$ nearest neighbors (best matches) are identified. In most instantiations of KNN, classification or prediction is based on a **majority vote** of the $k$ nearest neighbors.
> For example, if $k = 5$ and at least 3 out of the 5 nearest neighbors of an input datum are class A, then we would assign the new datum to class A.

For a more complex example, see the image below. Here, if $k = 1$, the green circle would be assigned to Class 1, since the nearest point is a blue square. However, if $k = 3$, the answer becomes Class 2, since the next two closest are both red triangles.


![knn.png](https://raw.githubusercontent.com/MedlyticsUniversal/Data/main/Images/knn.png)

## Quick example to illustrate a KNN:

In this section, we will use sklearn to build a k-NNA/KNN model for a dataset describing cars. Our goal will be to classify the cars into one of two categories: "cool" or "uncool." Clearly these are subjective terms, but that's okay: we will provide a manually classified training set.

For example, if we consider the variables _horsepower, number of seats,_ and _auto_ (manual = 0, automatic = 1), our manually classified training set might look like this:

- 150, 5, 0, uncool (2008 Honda Civic)  
- 320, 5, 0, cool (2011 Dodge Charger)
- 383, 3, 1, cool (1985 Chevy Blazer)
- 210, 7, 0, uncool (2001 Honda Odyssey)

Let's say we're trying to predict whether the 2017 Bugatti Veyron (1500hp, 2 seats, auto = 1) is cool or not. Our first step is to load the data into a Python structure.

In [1]:
import numpy as np
import pandas as pd

### Loading the data

In [16]:
cars_dict = {'2008 Honda Civic':    {'hp':150., 'seats':5., 'auto':0., 'cool':0}, 
             '2011 Dodge Charger' : {'hp':320., 'seats':5., 'auto':0., 'cool':1}, 
             '1985 Chevy Blazer':   {'hp':383., 'seats':3., 'auto':1., 'cool':1}, 
             '2001 Honda Odyssey':  {'hp':210., 'seats':7., 'auto':0., 'cool':0}, 
             '2017 Bugatti Veyron': {'hp':1500.,'seats':2., 'auto':1., 'cool':None}}

data = pd.DataFrame.from_dict(cars_dict,orient='index')
data

,hp,seats,auto,cool
2008 Honda Civic,150.0,5.0,0.0,0.0
2011 Dodge Charger,320.0,5.0,0.0,1.0
1985 Chevy Blazer,383.0,3.0,1.0,1.0
2001 Honda Odyssey,210.0,7.0,0.0,0.0
2017 Bugatti Veyron,1500.0,2.0,1.0,NaN


### Normalizing data and calculating distances

To determine the class of our Bugatti using NNA, we need to measure its "nearness" to other cars. One of the simplest metrics for this is the Euclidian distance, defined as the square root of the sum of the differences between each feature squared:

#### $\sqrt{(q_1-p_1)^2+(q_2-p_2)^2+ \ldots +(q_n-p_n)^2}$
#### $= \sqrt{\displaystyle\sum_{i=1}^{n} (q_i-p_i)^2 }$.

Some other approaches include Chi square distance and cosine distance. For further reading on distance functions, see this article: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4978658/.

Note that **by using Euclidian distance, we will be normalizing the data in a way that assumes all features are equally important**. Otherwise, values that are generally larger (such as horsepower) will end up having a larger impact on the distance. Unless we have some reason to believe that the larger features are more important, we want to balance features by **normalizing the data.** This will be helpful when we compare measurements of different units.

#### Min-Max Normalization

One way to normalize data is to rescale the range of data to [0, 1] (or [-1, 1]). To do this, we subtract the minimum value of a feature from each of its instances and then divide them by the range of the feature values.

### $ x_{scaled} = \dfrac {x - x_{min}} {x_{max}-x_{min}} $

Let's do this for our dataset:

In [17]:
# Normalizing each value by subtracting the minimum value in its row and then dividing by the range (max - min)

data_mmnorm = data

for i in ['hp','seats','auto']: # Remember that we don't normalize the label
    data_mmnorm[i] = (data[i] - min(data[i].values))/(max(data[i].values) - min(data[i].values))
    
data_mmnorm

,hp,seats,auto,cool
2008 Honda Civic,0.000000,0.6,0.0,0.0
2011 Dodge Charger,0.125926,0.6,0.0,1.0
1985 Chevy Blazer,0.172593,0.2,1.0,1.0
2001 Honda Odyssey,0.044444,1.0,0.0,0.0
2017 Bugatti Veyron,1.000000,0.0,1.0,NaN


However, a significant downside to min-max normalization is that it does not handle _outliers_ very well. For instance, normalizing the houses data below fixed the squished y-axis but was no good for the x-axis.

![min_max_problem.png](https://raw.githubusercontent.com/MedlyticsUniversal/Data/main/Images/min_max_problem.png)

#### Z-Score Normalization (Standardization)

Z-score normalization, also known as standardization, is a way of normalizing data that avoids this outlier issue. The strategy is to scale the values of a feature to have a mean of 0 and a standard deviation of 1:

### $ x_{scaled} = \dfrac {x - \mu} {\sigma} $
where $\mu$ is the mean value of the feature and $\sigma$ is the standard deviation of the feature.

If a value is equal to the mean of a feature, it will be scaled to 0. If it is below the mean, it will become a negative value, and if it is above the mean, it will be a positive value. Let's try this on the houses data from before.

See how the normalized data is less squished now? Also notice that the scales of the x-axis and y-axis are nearly the same—almost all points range from -2 to 2 on both axes. The only downside to note is that features won't be on the _exact_ same scale.

![z_score_norm.png](https://raw.githubusercontent.com/MedlyticsUniversal/Data/main/Images/z_score_norm.png)

As usual, a tool already exists to do this. Using sklearn, we can normalize data with an object called a `StandardScaler`. We will use this in the example below, but feel free to also read the documentation here:

http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.

In [18]:
# Normalize data by turning the mean into 0 and scaling to unit variance from each feature

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

data_znorm = pd.DataFrame.from_dict(cars_dict,orient='index')

for i in ['hp','seats','auto']:
    feature_data = data_znorm[i].values.reshape(-1, 1)
    scaler.fit(feature_data)
    data_znorm[i] = scaler.transform(feature_data)
    
data_znorm

,hp,seats,auto,cool
2008 Honda Civic,-0.724651,0.344124,-0.816497,0.0
2011 Dodge Charger,-0.384908,0.344124,-0.816497,1.0
1985 Chevy Blazer,-0.259004,-0.802955,1.224745,1.0
2001 Honda Odyssey,-0.604742,1.491202,-0.816497,0.0
2017 Bugatti Veyron,1.973305,-1.376494,1.224745,NaN


Now that the data is standardized, we can begin building our NNA algorithm. First, we will need a function that **calculates the Euclidean distance between two data points**. For our purposes, we will assume the data points will be stored in arrays of the same length.

In [20]:
# Distance function using formula for Euclidean distance

def euclidean_dist(datum1, datum2):
    within_sqrt = 0.0
    
    for g in range(datum1.shape[0]):
        within_sqrt += (datum1[g]- datum2[g]) ** 2
    
    distance = np.sqrt(within_sqrt)
    return(distance)

Next, we calculate the distance between our Bugatti and each of the other cars using the `euclidean_dist` function we defined. For the sake of testing, we do this twice: once for the normalized data `data_mmnorm` and once for the data standardized by sklearn `data_znorm`. Note that we will not input the classification column of cool/uncool to our distance function, which means taking a subset of each array.

In [21]:
# FYI: This is how you can call a specific row by name and sub-select features
bugatti = data.loc["2017 Bugatti Veyron"][["hp","seats","auto"]].values

import math

# Normalized data by dividing
print('Euclidean distances to 2017 Bugatti Veyron (V1)')
for car in range(len(data_mmnorm)):
    d = euclidean_dist(data_mmnorm.iloc[4, :3].values, 
                       data_mmnorm.iloc[car, :3].values)
    print('  {}: \t{:01.3f}'.format(data.index[car], d))

# Standardized data
print('\nEuclidean distances to 2017 Bugatti Veyron (V2)')
for car in range(len(data_znorm)):
    d = euclidean_dist(data_znorm.iloc[4, :3].values, 
                       data_znorm.iloc[car, :3].values)
    print('  {}: \t{:01.3f}'.format(data.index[car], d))

Euclidean distances to 2017 Bugatti Veyron (V1)
  2008 Honda Civic: 	1.536
  2011 Dodge Charger: 	1.457
  1985 Chevy Blazer: 	0.851
  2001 Honda Odyssey: 	1.707
  2017 Bugatti Veyron: 	0.000

Euclidean distances to 2017 Bugatti Veyron (V2)
  2008 Honda Civic: 	3.796
  2011 Dodge Charger: 	3.562
  1985 Chevy Blazer: 	2.305
  2001 Honda Odyssey: 	4.363
  2017 Bugatti Veyron: 	0.000


For the normalized data (with each feature scaled to [0, 1]), we get the following distances:

- Bugatti - Blazer = 0.851
- Bugatti - Charger = 1.457
- Bugatti - Civic = 1.536
- Bugatti - Odyssey = 1.707

For the standardized data (using sklearn's `StandardScaler`), we get the following distances:
- Bugatti - Blazer = 2.305
- Bugatti - Charger = 3.562
- Bugatti - Civic = 3.796
- Bugatti - Odyssey = 4.363

Notice that both techniques yielded the same order of cars from nearest to farthest.
> This is coincidental and unlikely to happen with larger or more varied datasets.

Since the distance between the Bugatti and Chevy Blazer is the smallest, if $k = 1$, we would classify the Bugatti as cool. However, if $k = 4$ there would no longer be a majority, and we would not be able to classify the Bugatti in either category without a tiebreaker protocol.

Generally speaking, _larger values of $k$ reduce noise but also make the boundaries between classes less distinct_. The best value of $k$ will vary by dataset.

# Example KNN 

Next we will see if we can use KNN on the Pima diabetes dataset.

## Loading data

In [22]:
url = "https://raw.githubusercontent.com/MedlyticsUniversal/Data/main/Week1/diabetes.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_csv(url, names=names)

# Dropping NaN rows
invalid = ['plas', 'pres', 'skin', 'test', 'mass']

for i in invalid:
    data[i].replace(to_replace=0, value=np.nan, inplace=True)
    
data = data.dropna(axis=0).reset_index(drop=True)

data.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
1,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
2,3,78.0,50.0,32.0,88.0,31.0,0.248,26,1
3,2,197.0,70.0,45.0,543.0,30.5,0.158,53,1
4,1,189.0,60.0,23.0,846.0,30.1,0.398,59,1


## Splitting data into training, validation, and testing sets

In [23]:
from sklearn.model_selection import train_test_split

# Columns we will use to make predictions with (features!). Feel free to play around with these:
X_cols = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age']

# Column that we want to predict (the labels)
y_col = 'class'

# 80-20 train-test split of datset
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(data[X_cols], data[y_col], test_size=test_size, random_state=0)

# Further split X and y of training into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=test_size, random_state=0)

## Normalizing data

In [24]:
scaler = StandardScaler()

for i in list(X_train):
    feature_data_train = X_train[i].values.reshape(-1, 1)
    scaler.fit(feature_data_train)
    X_train[i] = scaler.transform(feature_data_train)

for j in list(X_test):
    feature_data_test = X_test[j].values.reshape(-1, 1)
    scaler.fit(feature_data_test)
    X_test[j] = scaler.transform(feature_data_test)
    
for k in list(X_val):
    feature_data_val = X_val[k].values.reshape(-1, 1)
    scaler.fit(feature_data_val)
    X_val[k] = scaler.transform(feature_data_val)

## Using sklearn's KNN

Luckily for us, sklearn has some quick and easy functions for normalizing the data, finding Euclidean distances, training, and testing with [KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html). Try `k=5` to start.

In [25]:
from sklearn.neighbors import KNeighborsClassifier

# Creating model with sklearn's KNeighborsClassifier — after running these cells, play around with the parameter n!
knn = KNeighborsClassifier(n_neighbors=5)

# Training/fitting a model with training data
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [26]:
from sklearn.metrics import accuracy_score
import time

y_train_pred=knn.predict(X_train)
start = time.time()
predictions_fast = knn.predict(X_val)

print('Took {} seconds'.format(time.time() - start))
print("Training Accuracy is ", accuracy_score(y_train, y_train_pred)*100)
print("Validation Accuracy is ", accuracy_score(y_val,predictions_fast)*100)

Took 0.007447957992553711 seconds
Training Accuracy is  79.2
Validation Accuracy is  80.95238095238095


# Conclusions and Limitations

KNN provides a good baseline, machine learning classifier that is conceptually intuitive and easy to implement. One major advantage is that, by normalizing input data, a KNN algorithm can combine any number of features regardless of their original scales.

However, KNN can run into problems with more complex data sets:
- With additional dimensions, it can be harder to define meaningful distances.
- The testing phase can slow down significantly with large datasets since each point's distance is measured to every other point.
- "Majority votes" may be skewed if one classification is significantly more common than the others.
- There is no consideration for correlated features.

As a final note, there are other ways to instantiate KNNs, and other types of NNA beyond k-Nearest. For example, to solve the large majority problem, we could have used a **weighted voting system** where nearer neighbors' votes carry more weight.

_Note: The tutorials and graphs on normalization methods utilize content from [Codecademy](https://www.codecademy.com/article/normalization)._

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=67231ff1-13ba-4f0b-99c2-0fe76b4d3844' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>